In [1]:
from gmail import Gmail
from datetime import datetime
from setfit import SetFitModel
from bs4 import BeautifulSoup
import os
import pandas as pd
# from flan import JobTitleCompanyNameExtractor
from huggingchat import HFJobTitleCompanyNameExtractor
from tqdm import tqdm




In [2]:
email_clasification_model = SetFitModel.from_pretrained("./../models/job_email_classification_model_final")
job_stage_classification_model = SetFitModel.from_pretrained("./../models/job_stage_classification_model")
extractor = HFJobTitleCompanyNameExtractor()
def infer_one_sentence(model, sentence):
        predtext = [sentence]
        predicted_class = model(predtext)
        return str(predicted_class.numpy()[0])

/Users/akshit/Library/Caches/pypoetry/virtualenvs/talenttrove-mo4SAvxM-py3.10/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/akshit/Library/Caches/pypoetry/virtualenvs/talenttrove-mo4SAvxM-py3.10/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
def get_email_csv():
    gmail = Gmail(username="akshit.karanam99@gmail.com",password="uknf iafz dpna qyjb")
    gmail.authenticate()
    specified_date = datetime(2022, 9, 1)
    formatted_date = specified_date.strftime("%d-%b-%Y")
    ids = gmail.get_email_by_date(from_date=formatted_date)
    email_dict = gmail.parse_emails(ids[:10])
    emails_final_list = []
    for email in tqdm(email_dict):
        try:
            subject = (email['subject']).decode('utf-8')
        except:
            subject = email['subject']

        html = str(BeautifulSoup(email['body']).text)
        string_list = [s.strip() for s in str(html).split()]
        final_string = ' '.join(string_list)
        final_string_without_subject = ' '.join(string_list)
        final_string = "Subject: " + str(subject) +". Body: " + final_string
        stage = ""
        predicted_class = infer_one_sentence(email_clasification_model,final_string_without_subject)
        if predicted_class == "1":
            stage = infer_one_sentence(model=job_stage_classification_model,sentence=final_string)
            company = extractor.get_company(final_string)
            title = extractor.get_jobtitle(final_string)
            # print(stage)
            # print(company)
            # print(title)
            emails_final_list.append({'id':email['id'] , "date": email['date'],'title': title,"company":company,'stage':stage})
    df = pd.DataFrame(emails_final_list)        

    file_path = './../../data/applied_data/jobs_applied.csv'

    if os.path.exists(file_path):

        df.to_csv(file_path, mode='a', index=False, header=False)
    else:
        if df.empty:
            df = pd.DataFrame(columns=['id','date','title','company','stage'])
        df.to_csv(file_path, index=False)    

In [4]:
get_email_csv()